In [1]:
%config Completer.use_jedi = False

In [2]:
import pandas as pd
from datetime import datetime
from pykrige.ok import OrdinaryKriging
import numpy as np
import geopandas as gpd
import geobr
from scipy.spatial import KDTree
from shapely.geometry import Point
from scipy.spatial import KDTree
from statistics import mean
from windrose import WindroseAxes
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import xarray as xr
import itertools
plt.style.use("ggplot")

In [3]:
# Caminhos dos diretórios raiz
# general_path = 'C:/Users/cnalm/OneDrive/Hidroweb'
# general_path = 'D:/Dados_Nuvem/OneDrive/Hidroweb'
general_path = 'C:/Users/linde/OneDrive/Hidroweb'

In [4]:
df_total_hdf = pd.read_hdf(general_path + '/Consolidated Files/BRASIL_CLEANED_PARTITION_1961_1970.h5')
df_total_hdf = df_total_hdf.drop_duplicates(ignore_index = True).reset_index(drop = True)
df_total_hdf.tail(5)

,Date,Value,Code,Quality Index,Quality Label
15108362,1968-08-27,0.0,08351000,88.214887,Good Quality
15108363,1968-08-28,0.0,08351000,88.214887,Good Quality
15108364,1968-08-29,0.0,08351000,88.214887,Good Quality
15108365,1968-08-30,0.0,08351000,88.214887,Good Quality
15108366,1968-08-31,0.0,08351000,88.214887,Good Quality


In [5]:
df_info = pd.read_hdf(general_path + '/Consolidated Files/BRASIL_CLEANED_1961_2020_GAUGES.h5')
df_info.tail(5)

,Name,Code,City,State,Responsible,Latitude,Longitude
11022,ÁGUA FRIA,08460003,UIRAMUTA,RORAIMA,ANA,4.6428,-60.4964
11023,UIRAMUTA,08460004,UIRAMUTA,RORAIMA,ANA,4.5986,-60.1664
11024,NOVA ESPERANÇA/MARCO BV-8,08461000,PACARAIMA,RORAIMA,ANA,4.4883,-61.1297
11025,MISSÃO AUARIS - JUSANTE,08464001,BOA VISTA,RORAIMA,ANA,4.0031,-64.4431
11026,FAZENDA BANDEIRA BRANCA,08560000,UIRAMUTA,RORAIMA,ANA,4.6306,-60.4706


In [6]:
df_total = pd.merge(df_total_hdf, df_info, on = ['Code'], how ='left')
df_total = df_total[[
    'Date',
    'Value',
    'Code',
    'Name',
    'City',
    'State',
    'Responsible',
    'Latitude',
    'Longitude'
]]
df_total.tail(5)

,Date,Value,Code,Name,City,State,Responsible,Latitude,Longitude
15108362,1968-08-27,0.0,08351000,OIAPOQUE (CLEVELÂNDIA),OIAPOQUE,AMAPÁ,INMET,3.84,-51.82
15108363,1968-08-28,0.0,08351000,OIAPOQUE (CLEVELÂNDIA),OIAPOQUE,AMAPÁ,INMET,3.84,-51.82
15108364,1968-08-29,0.0,08351000,OIAPOQUE (CLEVELÂNDIA),OIAPOQUE,AMAPÁ,INMET,3.84,-51.82
15108365,1968-08-30,0.0,08351000,OIAPOQUE (CLEVELÂNDIA),OIAPOQUE,AMAPÁ,INMET,3.84,-51.82
15108366,1968-08-31,0.0,08351000,OIAPOQUE (CLEVELÂNDIA),OIAPOQUE,AMAPÁ,INMET,3.84,-51.82


In [7]:
df_coords = pd.read_hdf(general_path + "./Coords/BRASIL_GRID_0_DOT_25_DEGREE.h5")
df_coords_temp = df_coords[['Latitude', 'Longitude']]
df_coords_temp

,Latitude,Longitude
0,-33.625,-53.375
1,-33.375,-53.375
2,-33.375,-53.125
3,-33.125,-53.125
4,-33.125,-52.875
...,...,...
11345,4.875,-60.375
11346,4.875,-60.125
11347,5.125,-60.625
11348,5.125,-60.375


In [8]:
# Function for IDW interpolation

# In the IDW methodology, each of the nearest stations
# selected for the interpolation at a query point is weighted
# (Wk) by Wk=d(k)−p, where d is the distance of station k and
# the specified query point. The p values is the power
# parameter that we use p = 2, as suggested by Ly et al.
# (2011) and Xavier et al. (2016).
# (Dirks et al., 1998), Goovaert (2000) and Lloyd (2005) 

def idw_interpolation(row, p=2):
    # Find the indices and distances of the 5 nearest stations 
    step_size = 0.25 / 4

    start_lat = row['Latitude'] - 0.125
    end_lat = row['Latitude'] + 0.125 + step_size  # Add step_size to include the endpoint
    generated_latitudes = [round(start_lat + i * step_size, 6) for i in range(int((end_lat - start_lat) / step_size))]

    start_lon = row['Longitude'] - (0.25 / 2)
    end_lon = row['Longitude'] + (0.25 / 2) + step_size  # Add step_size to include the endpoint
    generated_longitudes = [round(start_lon + i * step_size, 6) for i in range(int((end_lon - start_lon) / step_size))]

    interpolated_value_avg = []

    for lat in generated_latitudes:
        for lon in generated_longitudes:
            distances, indices = kdtree.query([lat, lon], k=5)
            max_distance = 0
            if max(distances) >= max_distance:
                max_distance = max(distances)
            # Compute the inverse distance weights
            weights = 1 / (distances + 1e-6) ** p  # Adding a small value to prevent division by zero
    
            # Get the values at the nearest stations
            values = df_temp.iloc[indices]['Value'].values
    
            # Calculate the weighted average
            interpolated_value = np.sum(weights * values) / np.sum(weights)
            interpolated_value_avg.append(interpolated_value)
    # print("max distance", max_distance)
            
    interpolated_value_final = mean(interpolated_value_avg)
    # if interpolated_value_final > 0:
        # print('interpolated_value_avg', interpolated_value_avg)
        # print('interpolated_value_final', interpolated_value_final)
        # print('coordinates', row['Latitude'], row['Longitude'])
        # print("generated_latitudes", generated_latitudes)
        # print("generated_longitudes", generated_longitudes)
    return interpolated_value_final

In [9]:
# ref_date = datetime.strptime('2022-11-05', '%Y-%m-%d')
df_date_list = pd.DataFrame(df_total['Date'].drop_duplicates().sort_values()).query("Date >= '1965-12-26'")
date_list = df_date_list['Date'].tolist()
### TESTING | TESTING | TESTING | TESTING | TESTING | TESTING | TESTING
# date_list = date_list[0:5]
### TESTING | TESTING | TESTING | TESTING | TESTING | TESTING | TESTING
date_list

[Timestamp('1965-12-26 00:00:00'),
 Timestamp('1965-12-27 00:00:00'),
 Timestamp('1965-12-28 00:00:00'),
 Timestamp('1965-12-29 00:00:00'),
 Timestamp('1965-12-30 00:00:00'),
 Timestamp('1965-12-31 00:00:00'),
 Timestamp('1966-01-01 00:00:00'),
 Timestamp('1966-01-02 00:00:00'),
 Timestamp('1966-01-03 00:00:00'),
 Timestamp('1966-01-04 00:00:00'),
 Timestamp('1966-01-05 00:00:00'),
 Timestamp('1966-01-06 00:00:00'),
 Timestamp('1966-01-07 00:00:00'),
 Timestamp('1966-01-08 00:00:00'),
 Timestamp('1966-01-09 00:00:00'),
 Timestamp('1966-01-10 00:00:00'),
 Timestamp('1966-01-11 00:00:00'),
 Timestamp('1966-01-12 00:00:00'),
 Timestamp('1966-01-13 00:00:00'),
 Timestamp('1966-01-14 00:00:00'),
 Timestamp('1966-01-15 00:00:00'),
 Timestamp('1966-01-16 00:00:00'),
 Timestamp('1966-01-17 00:00:00'),
 Timestamp('1966-01-18 00:00:00'),
 Timestamp('1966-01-19 00:00:00'),
 Timestamp('1966-01-20 00:00:00'),
 Timestamp('1966-01-21 00:00:00'),
 Timestamp('1966-01-22 00:00:00'),
 Timestamp('1966-01-

In [10]:
### TESTING | TESTING | TESTING | TESTING | TESTING | TESTING | TESTING

# date_test = '2000-01-04'
# df_temp_2 = df_total[(df_total['Date'] == date_test)]
# df_temp_3 = df_temp_2['State'].drop_duplicates()
# print(len(df_temp_3),"estados")
# date_list = [datetime.strptime(date_str, '%Y-%m-%d') for date_str in [date_test]]
# date_list

### TESTING | TESTING | TESTING | TESTING | TESTING | TESTING | TESTING

In [11]:
for ref_date in date_list:
    # print(ref_date)
    df_temp = df_total[(df_total['Date'] == ref_date)]
    # print(df_temp)
    """
    sorted_df = df_temp.sort_values(by='Value', ascending=False)
    geometry = [Point(xy) for xy in zip(sorted_df['Longitude'], sorted_df['Latitude'])]
    gdf_gauges = gpd.GeoDataFrame(sorted_df, geometry=geometry)
    # print(gdf_gauges)
    # SHP = gpd.read_file(general_path + '/SHP/Brazilian_Border.shp')
    state_data = geobr.read_state(year=2019)  # Adjust the year if needed
    state_data.crs = "EPSG:4326"
    state_data["name_state"] = state_data["name_state"].str.upper()
    # print(state_data.head(5))
    fig, ax = plt.subplots(figsize = (10, 7))
    # SHP.plot(ax = ax, color = 'w')
    # North pointer
    x, y, arrow_length = 0.95, 0.95, 0.1
    ax.annotate('N', xy=(x, y), xytext=(x, y-arrow_length),
                arrowprops=dict(facecolor='black', width=5, headwidth=15),
                ha='center', va='center', fontsize=18,
                xycoords=ax.transAxes)
    state_data.plot(ax = ax, alpha=0.75, edgecolor = 'black', color = 'white')
    gdf_gauges.plot(ax=ax, column='Value', legend=True, cmap='plasma', s = 2)
    # Set labels and title
    ax.set_xlabel('Longitude')
    ax.set_ylabel('Latitude')
    ax.set_title('HidroWeb ground data:' + str(ref_date.date()))
    # Set legend label
    # plt.legend(title=f'Precipitation in mm')
    # Show the plot
    plt.tight_layout()
    plt.show()
    plt.close()
    """
    # Create a KDTree from latitudes and longitudes
    locations = df_temp[['Latitude', 'Longitude']].values
    kdtree = KDTree(locations)
    df_precip = df_coords_temp.copy(deep=True)
    df_precip['Precipitation'] = df_precip.apply(idw_interpolation, axis=1)
    df_precip['Date'] = ref_date
    """
    # Create a geometry column by combining Latitude and Longitude into a Point object
    geometry = [Point(xy) for xy in zip(df_precip['Longitude'], df_precip['Latitude'])]
    # Create a GeoDataFrame using the geometry column and other attributes
    gdf_rainfall = gpd.GeoDataFrame(df_precip, geometry=geometry)
    # Plotting the GeoPandas GeoDataFrame with a color scale based on 'Interpolated_Value'
    fig, ax = plt.subplots(figsize=(10, 8))  # Adjust the figure size if needed
    # Plotting the points with a color scale based on the 'Interpolated_Value' column
    ax.annotate('N', xy=(x, y), xytext=(x, y-arrow_length),
                arrowprops=dict(facecolor='black', width=5, headwidth=15),
                ha='center', va='center', fontsize=18,
                xycoords=ax.transAxes)
    state_data.plot(ax = ax, alpha=0.75, edgecolor = 'black', color = 'white')
    gdf_rainfall.plot(ax=ax, column='Precipitation', legend=True, cmap='plasma')
    # Adding title and labels
    plt.title('Interpolated data (IDW):' + str(ref_date.date()))
    plt.xlabel('Longitude')
    plt.ylabel('Latitude')
    # Show the plot
    plt.tight_layout()
    plt.show()
    plt.close()
    """
    df_precip.set_index(['Latitude', 'Longitude', 'Date'], inplace=True)
    ds = xr.Dataset.from_dataframe(df_precip)
    # Set attributes for precipitation
    ds['Precipitation'].attrs['units'] = 'mm'
    ds.to_netcdf(general_path + '/Consolidated Files/NetCDF/IDW/precipitation_idw_'+ str(ref_date.date()) +'.nc')
    print(general_path + '/Consolidated Files/NetCDF/IDW/precipitation_idw_'+ str(ref_date.date()) +'.nc')
    # ds_data = ds['Precipitation'].isel(Date=0)
    # fig, ax = plt.subplots(figsize = (7, 5))
    # ds_data.plot(ax = ax, cmap='rainbow')
    # plt.tight_layout()
    # plt.show()
    # plt.savefig(general_path + '/Figures/IDW/precipitation_idw' + str(ref_date.date()) + '.png', format='png', dpi=300, transparent=False, bbox_inches=None)
    # plt.close()

C:/Users/linde/OneDrive/Hidroweb/Consolidated Files/NetCDF/IDW/precipitation_idw_1965-12-26.nc
C:/Users/linde/OneDrive/Hidroweb/Consolidated Files/NetCDF/IDW/precipitation_idw_1965-12-27.nc
C:/Users/linde/OneDrive/Hidroweb/Consolidated Files/NetCDF/IDW/precipitation_idw_1965-12-28.nc
C:/Users/linde/OneDrive/Hidroweb/Consolidated Files/NetCDF/IDW/precipitation_idw_1965-12-29.nc
C:/Users/linde/OneDrive/Hidroweb/Consolidated Files/NetCDF/IDW/precipitation_idw_1965-12-30.nc
C:/Users/linde/OneDrive/Hidroweb/Consolidated Files/NetCDF/IDW/precipitation_idw_1965-12-31.nc
C:/Users/linde/OneDrive/Hidroweb/Consolidated Files/NetCDF/IDW/precipitation_idw_1966-01-01.nc
C:/Users/linde/OneDrive/Hidroweb/Consolidated Files/NetCDF/IDW/precipitation_idw_1966-01-02.nc
C:/Users/linde/OneDrive/Hidroweb/Consolidated Files/NetCDF/IDW/precipitation_idw_1966-01-03.nc
C:/Users/linde/OneDrive/Hidroweb/Consolidated Files/NetCDF/IDW/precipitation_idw_1966-01-04.nc
C:/Users/linde/OneDrive/Hidroweb/Consolidated File